In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import pearsonr, spearmanr, kendalltau
import math
import plotly.io as pio
pio.renderers.default = "plotly_mimetype"

# Seed for reproducibility
np.random.seed(42)

# Number of datapoints for each scatter plot
n_points = 400

# Function to generate synthetic data for brain volume vs. cognitive performance
def generate_synthetic_data(volume_mean, volume_std, score_mean, score_std, correlation, n_points):
    volume = np.random.normal(volume_mean, volume_std, n_points)
    volume_jitter = np.random.normal(0, volume_std * 1.5, n_points)  # Add jitter to volume
    noise = np.random.normal(0, score_std * 10.5, n_points)  # Increase noise for jitter in score
    score = score_mean +  correlation * (volume - volume_mean) + noise
    score[-58:] = [x * 5 for x in score[-58:]]
    return volume + volume_jitter, score

# Function to generate additional metrics
def generate_additional_metrics(volume, score, volume_std, score_std, n_points):
    # Hypothetical Brain Density: Based on volume with added noise
    density = volume / (volume_std * np.random.uniform(0.8, 1.2, n_points))
    noise = np.random.normal(0, score_std * 0.01, n_points)  # Increase noise for jitter in score
    
    # Hypothetical Neural Efficiency: Positively correlated with score but with noise
    efficiency = score / (score_std * np.random.uniform(1.2, 11.7, n_points))
    
    return density, efficiency

# Generate synthetic data for 10 scatter plots
data_dict = {}

# Scatter plot 7: Temporal Cortex Volume vs. Language Ability
data_dict["Temporal_Language"] = generate_synthetic_data(500, 400, 75, 9, 0.20, n_points)

df_dict = {key: pd.DataFrame({'Volume': value[0], 'Score': value[1]}) for key, value in data_dict.items()}

# Apply additional metrics to all regions
for key, df in df_dict.items():
    volume_std = np.std(df['Volume'])
    score_std = np.std(df['Score'])
    
    # Generate new metrics for Brain Density and Neural Efficiency
    density, efficiency = generate_additional_metrics(df['Volume'], df['Score'], volume_std, score_std, n_points)
    
    # Add the new metrics to the DataFrame
    df['Density'] = density
    df['Efficiency'] = efficiency

def plot_correlation(df_dict, key, x_col, y_col, title_prefix, color='blue', marker_size=10, marker_opacity=0.7):
    df = df_dict[key]
    
    # Calculate Spearman correlation
    corr, _ = kendalltau(df[x_col], df[y_col])
    
    # Create scatter plot with customized marker colors, size, and opacity
    fig = px.scatter(
        df, 
        x=x_col, 
        y=y_col, 
        title=f'{title_prefix} (Kendall tau: {corr:.2f})', 
        trendline='ols',
        color_discrete_sequence=[color],  # Custom marker color
        template="ggplot2",
        marginal_y="violin",
    )
    
    # Update marker style
    # fig.update_traces(marker=dict(size=marker_size, opacity=marker_opacity))
    
    # Update figure layout
    fig.update_layout(height=600)
    
    return fig


In [ ]:
#| label: fig1cell 

fig1 = plot_correlation(df_dict, "Temporal_Language", 'Volume', 'Score', 'Temporal', color='black', marker_size=10, marker_opacity=0.7)
fig1.show()

In [ ]:
#| label: fig2cell

fig2 = plot_correlation(df_dict, "Temporal_Language", 'Density', 'Score', 'Temporal', color='pink', marker_size=10, marker_opacity=0.7)
fig2.show()

In [ ]:
#| label: fig3cell

fig2 = plot_correlation(df_dict, "Temporal_Language", 'Efficiency', 'Score', 'Amygdala', color="orange", marker_size=10, marker_opacity=0.7)
fig2.show()
